In [ ]:
import os
import sys
import pandas as pd
import numpy as np
module_path = os.path.abspath(os.path.join('../../data/..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%matplotlib inline

In [2]:
businessPrepped = pd.read_pickle('../../data/prep/businessPrepped.pkl')

In [3]:
businessPrepped.head()

,business_id,name,state,stars,review_count,is_open,postal_code,categories,checkins,tipcount
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",AZ,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",PA,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",AZ,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",AZ,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0


In [4]:

#Returns NaN for closed businesses, and the current star rating for open businesses.
def openStars(row):
    if row['is_open'] == 0:
       return np.nan
    return row['stars']

def closedStars(row):
    if row['is_open'] == 1:
       return np.nan
    return row['stars']


In [5]:
#businessPrepped['stars'] = businessPrepped['stars'].astype(float)
businessPrepped['open_stars'] = businessPrepped.apply(lambda row : openStars(row), axis=1)
businessPrepped['closed_stars'] = businessPrepped.apply(lambda row : closedStars(row), axis=1)
businessPrepped.head()

,business_id,name,state,stars,review_count,is_open,postal_code,categories,checkins,tipcount,open_stars,closed_stars
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",AZ,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0,4.0,NaN
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",PA,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0,3.0,NaN
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",AZ,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0,1.5,NaN
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",AZ,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0,NaN,3.0
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0,3.5,NaN


In [6]:

#Returns "" for closed businesses, and the current current category for open businesses.
def openCategories(row):
    if row['is_open'] == 0:
       return ""
    return row['categories']

def closedCategories(row):
    if row['is_open'] == 1:
       return ""
    return row['categories']



In [7]:
businessPrepped['closed_categories'] = businessPrepped.apply(lambda row : closedCategories(row), axis=1)
businessPrepped['open_categories'] = businessPrepped.apply(lambda row : openCategories(row), axis=1)
businessPrepped.head()

,business_id,name,state,stars,review_count,is_open,postal_code,categories,checkins,tipcount,open_stars,closed_stars,closed_categories,open_categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",AZ,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0,4.0,NaN,,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",PA,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0,3.0,NaN,,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",AZ,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0,1.5,NaN,,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",AZ,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0,NaN,3.0,Sporting Goods;Shopping,
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",OH,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0,3.5,NaN,,American (New);Nightlife;Bars;Sandwiches;Ameri...


Let's try create area profiles; #businesses, #open, #reviews, avg. stars, avg. starsOpen,category dictionary

starsOpen : Average Stars of Open Businesses in an area.

In [8]:
def categoryAgg(categoryColumn):
    areaCategories= {}
    for value in categoryColumn:
        categoryList = value.lower().split(";")
        for category in categoryList:
            if category in areaCategories:
                areaCategories[category] += 1
            elif category != "":
                areaCategories[category] = 1
    return areaCategories

In [9]:
#Operates on 'name' column and returns a dictionary counting the number of different chains.
def chainAgg(chainColumn):
    #Self-determined list of chains based on their appearances in dataset.
    chainList = ['"Starbucks"', '"McDonald"\'s','"Subway"', '"Pizza Hut"','"Taco Bell"','"Burger King"','"Walgreens"','"Wendy\'s"',\
                '"The UPS Store"', '"Tim Hortons"']
    areaChains= {}
    for chain in chainColumn:
        if chain in chainList:
            if chain in areaChains:
                areaChains[chain] += 1
            else :
                areaChains[chain] = 1
    return areaChains

In [10]:
#make second stars column for std. deviation of ratings
businessPrepped['std.dev_stars'] = businessPrepped['stars']

stateProfileGroups = businessPrepped.groupby('state').agg({'business_id':'count', 'is_open':sum, 'review_count':'mean',\
                                                               'checkins':'mean', 'tipcount':'mean',\
                                                               'open_stars':'mean', 'closed_stars':'mean',\
                                                              'std.dev_stars':"std" , 'open_categories':categoryAgg, \
                                              'closed_categories':categoryAgg, 'name':chainAgg})

Let's transfer these groups into a dataframe.

In [13]:
stateProfileDf = pd.DataFrame(stateProfileGroups)
stateProfileDf.reset_index(inplace = True)
stateProfileDf.head(100)

,state,business_id,is_open,review_count,checkins,tipcount,open_stars,closed_stars,std.dev_stars,open_categories,closed_categories,name
0,01,10,10,4.000000,9.000000,1.500000,3.700000,NaN,0.788811,"{'restaurants': 10, 'italian': 2, 'pizza': 3, ...",{},{}
1,3,1,1,4.000000,5.000000,0.000000,5.000000,NaN,NaN,"{'austrian': 1, 'restaurants': 1, 'beer garden...",{},{}
2,30,1,1,3.000000,0.000000,0.000000,3.000000,NaN,NaN,"{'restaurants': 1, 'french': 1}",{},{}
3,6,3,3,3.333333,4.666667,0.666667,3.666667,NaN,1.154701,"{'austrian': 1, 'restaurants': 1, 'nightlife':...",{},{}
4,AB,1,1,3.000000,0.000000,0.000000,5.000000,NaN,NaN,"{'event planning & services': 1, 'private tuto...",{},{}
5,ABE,3,3,4.000000,12.333333,0.666667,4.000000,NaN,0.866025,"{'restaurants': 1, 'coffee & tea': 1, 'food': ...",{},{}
6,AK,1,1,22.000000,14.000000,2.000000,2.500000,NaN,NaN,"{'restaurants': 1, 'fast food': 1, 'burgers': 1}",{},{}
7,AL,1,1,10.000000,7.000000,4.000000,5.000000,NaN,NaN,"{'hair removal': 1, 'event planning & services...",{},{}
8,AR,2,2,15.000000,4.000000,1.000000,3.250000,NaN,2.474874,"{'festivals': 1, 'arts & entertainment': 1, 'h...",{},{}
9,AZ,52214,44045,31.173498,104.974087,6.551519,3.762606,3.546028,1.060034,"{'dentists': 1362, 'general dentistry': 1041, ...","{'sporting goods': 125, 'shopping': 1409, 'boo...","{'""The UPS Store""': 99, '""Taco Bell""': 109, '""..."


We can see that there a lot of states with very few businesses, if we take a closer look we can see that some of them were not expected to be in our database. For example, 'NYK' is North Yorkshire in England.


In [16]:
businessPrepped[businessPrepped['state'] == 'NYK']

,business_id,name,state,stars,review_count,is_open,postal_code,categories,checkins,tipcount,open_stars,closed_stars,closed_categories,open_categories,std.dev_stars
1009,qzA-93v2lwf9qnCOPyWiUQ,"""Scarborough Deaf Club""",NYK,4.0,4,1,YO11,Public Services & Government,0.0,0.0,4.0,NaN,,Public Services & Government,4.0
1792,PjzIBRm4pxV_mivCZtmZlw,"""Duke Of Wellington""",NYK,5.0,3,1,YO21,Hotels & Travel;Event Planning & Services;Hotels,0.0,0.0,5.0,NaN,,Hotels & Travel;Event Planning & Services;Hotels,5.0
1841,PX9ceQzCEX5AEn73_UlZ2g,"""the coffee bean""",NYK,4.0,5,1,YO12,Food;Coffee & Tea,0.0,0.0,4.0,NaN,,Food;Coffee & Tea,4.0
5755,Rv7TH2KWTUXpjFPwTTp4Xg,"""Bonnet & Sons""",NYK,3.5,3,1,YO11,Coffee & Tea;Food;Cafes;Restaurants,0.0,1.0,3.5,NaN,,Coffee & Tea;Food;Cafes;Restaurants,3.5
7447,zIYzpJ70IEgQI_kAZF0a4g,"""Nippy Taxis""",NYK,2.5,3,1,YO11,Hotels & Travel;Transportation;Taxis,0.0,0.0,2.5,NaN,,Hotels & Travel;Transportation;Taxis,2.5
8692,qFBEXTIBrAsTsAGaMXOTEw,"""Mainline Menswear""",NYK,3.5,3,1,YO11,Shopping;Men's Clothing;Fashion,0.0,0.0,3.5,NaN,,Shopping;Men's Clothing;Fashion,3.5
9450,Wijq97TO27mhi_OG-xaW-w,"""Golden Grid Restaurant""",NYK,3.5,3,1,YO11,Seafood;Restaurants,1.0,0.0,3.5,NaN,,Seafood;Restaurants,3.5
10126,zadkJh0PvDYhP8sw0E8hLQ,"""Scarborough Railway Station""",NYK,3.0,3,1,YO11,Train Stations;Hotels & Travel,9.0,0.0,3.0,NaN,,Train Stations;Hotels & Travel,3.0
10391,2LdSbmZnJJS5DiwvaAp4Yw,"""Saba Thai Restaurant""",NYK,3.5,6,1,nan,Shopping,0.0,0.0,3.5,NaN,,Shopping,3.5
11981,xcPmvEfUMlgUFfLmrF1tQA,"""Pattisons""",NYK,3.5,3,1,YO12,Shopping;Flowers & Gifts;Florists,0.0,0.0,3.5,NaN,,Shopping;Flowers & Gifts;Florists,3.5


On investigation of 'NYK' we can see that the postal codes cover only a small region of the NYK area. As a result of this coupled with the overall small number of businesses, we do not feel this is representative of a state.

<p>We'll assume a similar situation for most states with low numbers of businesses, to avoid these states we'll focus our state study on the US and Canada</p>

In [20]:
stateProfileDf['%closed'] = (stateProfileDf['business_id'] - stateProfileDf['is_open']) / stateProfileDf['business_id']

We will now alter the column titles to accurately reflect their contents. We will also rearrange the columns as desired.

In [21]:
stateProfileDf.columns=['state','num_businesses','num_open','num_reviews','num_checkins','num_tips','open_stars','closed_stars',\
                        'std.dev_stars','open_categories','closed_categories','chains','%closed']

#Now Rearrange Columns
stateProfileDf = stateProfileDf[['state', 'num_businesses','num_open','%closed','num_reviews','num_checkins','num_tips','open_stars','closed_stars',\
                             'std.dev_stars','open_categories','closed_categories','chains']]

stateProfileDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 13 columns):
state                67 non-null object
num_businesses       67 non-null int64
num_open             67 non-null int64
%closed              67 non-null float64
num_reviews          67 non-null float64
num_checkins         67 non-null float64
num_tips             67 non-null float64
open_stars           65 non-null float64
closed_stars         27 non-null float64
std.dev_stars        40 non-null float64
open_categories      67 non-null object
closed_categories    67 non-null object
chains               67 non-null object
dtypes: float64(7), int64(2), object(4)
memory usage: 6.9+ KB


In [27]:
stateProfileSelection = stateProfileDf.sort_values('num_businesses', ascending = False).head(12)

In [28]:
stateProfileSelection.to_pickle('../../data/analysis/stateProfiles.pkl')
stateProfileSelection.to_csv('../../data/analysis/stateProfiles.csv')